# Daya Bay model in Jupyter notebook

## Preparation

Make sure that you run notebook in the same directory where you have `./data` for model.


In [ ]:
%matplotlib

In [ ]:
from dag_modelling.tools.logger import set_verbosity

from dgm_dayabay_dev.models import load_model
from dgm_fit.iminuit_minimizer import IMinuitMinimizer

set_verbosity(1)

## Model parameters

`MODEL_*` variables describe possible parameters of the loaded model. To get more info about parameters, just check documentation of `load_model`.

## Next sample

Method `next_sample()` is called to generate a new sample of pseudo data.

In [ ]:
MODEL_VERSION = "latest"
MODEL_SOURCE_TYPE = "npz"
MODEL_ANUE_SPEC = "exponential"
MODEL_SEED = 0
MODEL_MC_DATA = "asimov"
CONCATENATION_MODE = "detector"

model = load_model(
    MODEL_VERSION,
    spectrum_correction_interpolation_mode=MODEL_ANUE_SPEC,
    monte_carlo_mode=MODEL_MC_DATA,
    seed=MODEL_SEED,
    concatenation_mode=CONCATENATION_MODE,
)

model.next_sample()
model.switch_data("asimov")

storage = model.storage
parameters_free = storage["parameters.free"]
parameters_constrained = storage["parameters.constrained"]
statistic = storage["outputs.statistic"]

In [ ]:
chi2_type = "stat.chi2cnp"

parameters_surprob = dict(parameters_constrained["survival_probability"].walkjoineditems())
parameters_surprob.update(dict(parameters_free["survival_probability"].walkjoineditems()))
parameters_surprob_spec = parameters_surprob.copy()
parameters_surprob_spec.update(dict(parameters_free["neutrino_per_fission_factor"].walkjoineditems()))

In [ ]:
chi2 = statistic[chi2_type]

minimizer_spec = IMinuitMinimizer(chi2, parameters=parameters_surprob_spec)

In [ ]:
minimizer_spec.push_initial_values()
minimizer_spec.fit()

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

def visualization(args):
    plt.plot(model.storage["outputs.eventscount.final.concatenated.selected"].data, label="Model")
    data = model.storage["outputs.data.pseudo.self"].data
    x = np.arange(data.shape[0])
    plt.errorbar(x=x, y=data, xerr=0.5, yerr=data**.5, label="DATA", linestyle="none")
    plt.legend()

In [ ]:
minimizer_spec._minimizer.interactive(plot=visualization)